In [29]:
import numpy as np
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta
import time

In [30]:
def get_date_rage(release):
    year, month, day = release.split('-')
    release_3ago = (datetime(int(year), int(month), int(day)) - relativedelta(years=3)).strftime('%Y-%m-%d')
    return [release_3ago, release]

### 파일 이름
|name|desc|
|:---|:---|
|file_name_movie_df|가져올 영화 정보 파일|
|file_name_movie_actor_df|수집해야 할 영화의 주연 배우 2명의 배우 코드|
|file_name_actor_movie|배우가 주연으로 출연한 모든 출연작|
|directory_movie_info|영화의 관람객수와 개봉일을 저장할 디렉토리|
|file_name_movie_info_total|분산되어 있던 파일 병합|

In [47]:
# 파일이름
file_name_movie_df = 'test-data-netflix.xlsx'
file_name_movie_actor_df = 'netflix_preprocessed_data/movie_actor_df.csv'
file_name_actor_movie = 'netflix_preprocessed_data/actor_movie.csv'
directory_movie_info = './netflix_preprocessed_data/movie_info/'
file_name_movie_info_total = 'netflix_preprocessed_data/movie_info_total.csv'
file_name_merge_movie_info_df = 'netflix_preprocessed_data/movie_actor_power.csv'

In [33]:
actor_movie = pd.read_csv(file_name_actor_movie, index_col=0)
actor_movie

,actor_code,movie_name,movie_code
0,20201026,승리호(SPACE SWEEPERS),20192662
1,20201026,외계+인,20208446
2,20201026,리틀 포레스트(Little Forest),20170841
3,20201026,1987(1987: When the Day Comes),20170590
4,20201026,아가씨(The Handmaiden),20144444
...,...,...,...
190,10069179,좋은 친구들(Confession),20148745
191,10069179,결혼전야(Marriage Blue),20136061
192,10069179,나는 왕이로소이다(I am the King),20124080
193,10069179,키친(The Naked Kitchen),20090022


In [34]:
movie_actor_df = pd.read_csv(file_name_movie_actor_df, index_col=0)
movie_actor_df = movie_actor_df.fillna(0)
movie_actor_df = movie_actor_df.astype({'actor2_code': 'int'})
movie_actor_df

,movie_name,code,actor1_code,actor2_code
0,오징어 게임,0.0,10057315,20305129
1,킹덤,0.0,10069179,10019065
2,보건교사 안은영,0.0,10062071,20279929
3,옥자,0.0,10081721,10063816
4,승리호,0.0,10037229,20201026


In [35]:
movie_df = pd.read_excel(file_name_movie_df).loc[:,['movie_name','release']]

merge_movie_info_df = pd.merge(left = movie_actor_df , right = movie_df, how = "left", on = "movie_name")
merge_movie_info_df['actor_power'] = np.nan
merge_movie_info_df=merge_movie_info_df.astype({'release': 'str'})
merge_movie_info_df

,movie_name,code,actor1_code,actor2_code,release,actor_power
0,오징어 게임,0.0,10057315,20305129,2021-09-17,NaN
1,킹덤,0.0,10069179,10019065,2019-01-25,NaN
2,보건교사 안은영,0.0,10062071,20279929,2020-09-25,NaN
3,옥자,0.0,10081721,10063816,2017-05-19,NaN
4,승리호,0.0,10037229,20201026,2021-02-05,NaN


In [36]:
movie_info_total = pd.read_csv(file_name_movie_info_total, index_col=0)

In [37]:
merge_movie_df = pd.merge(left = actor_movie , right = movie_info_total, how = "left", on = "movie_code")

In [51]:
drop_index = merge_movie_df[merge_movie_df['release'] == '해당정보없음'].index
merge_movie_df = merge_movie_df.drop(drop_index)
merge_movie_df=merge_movie_df.reset_index(drop=True)
merge_movie_df.to_csv('./test.csv', encoding='utf-8-sig')
merge_movie_df

,actor_code,movie_name,movie_code,release,acc_audience_num
0,20201026,리틀 포레스트(Little Forest),20170841,2018-02-28,1505560
1,20201026,1987(1987: When the Day Comes),20170590,2017-12-27,7231638
2,20201026,아가씨(The Handmaiden),20144444,2016-06-01,4287839
3,20201026,문영(Moon young),20160881,2017-01-12,7761
4,10057315,다만 악에서 구하소서 파이널컷(DELIVER US FROM EVIL FINALCUT),20209304,2020-10-28,37961
...,...,...,...,...,...
136,10069179,좋은 친구들(Confession),20148745,2014-07-10,407210
137,10069179,결혼전야(Marriage Blue),20136061,2013-11-21,1214836
138,10069179,나는 왕이로소이다(I am the King),20124080,2012-08-08,790181
139,10069179,키친(The Naked Kitchen),20090022,2009-02-05,142336


In [16]:
for i in range(len(merge_movie_info_df)):
    movie_code = merge_movie_info_df['code'][i]
    actor1_code = merge_movie_info_df['actor1_code'][i]
    actor2_code = merge_movie_info_df['actor2_code'][i]
    release = list(merge_movie_info_df.loc[merge_movie_info_df['code']==movie_code, 'release'].values)[0]

    date_range = get_date_rage(release)
    filtered_movie_list = merge_movie_df[
        ((merge_movie_df['actor_code'] == actor1_code) |
        (merge_movie_df['actor_code'] == actor2_code)) &
        ((merge_movie_df['release'] >= date_range[0]) & (merge_movie_df['release'] <= date_range[1]))
    ]

    audience_mean = np.mean(filtered_movie_list['acc_audience_num'])
    merge_movie_info_df.loc[merge_movie_info_df['code']==movie_code, 'actor_power'] = audience_mean

    

In [45]:
# 넷플릭스용
for i in range(len(merge_movie_info_df)):
    movie_name = merge_movie_info_df['movie_name'][i]
    actor1_code = merge_movie_info_df['actor1_code'][i]
    actor2_code = merge_movie_info_df['actor2_code'][i]
    release = merge_movie_info_df['release'][i]

    date_range = get_date_rage(release)
    filtered_movie_list = merge_movie_df[
        ((merge_movie_df['actor_code'] == actor1_code) |
        (merge_movie_df['actor_code'] == actor2_code)) &
        ((merge_movie_df['release'] >= date_range[0]) & (merge_movie_df['release'] <= date_range[1]))
    ]

    audience_mean = np.mean(filtered_movie_list['acc_audience_num'])
    merge_movie_info_df.loc[merge_movie_info_df['movie_name']==movie_name, 'actor_power'] = audience_mean

In [43]:
merge_movie_info_df

,movie_name,code,actor1_code,actor2_code,release,actor_power
0,오징어 게임,0.0,10057315,20305129,2021-09-17,1.837094e+06
1,킹덤,0.0,10069179,10019065,2019-01-25,6.219300e+06
2,보건교사 안은영,0.0,10062071,20279929,2020-09-25,3.041107e+06
3,옥자,0.0,10081721,10063816,2017-05-19,5.474752e+05
4,승리호,0.0,10037229,20201026,2021-02-05,1.505560e+06


In [48]:
merge_movie_info_df.to_csv(file_name_merge_movie_info_df, encoding='utf-8-sig')